In [1]:
import sympy as sym
from sympy.utilities.lambdify import lambdastr
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from IPython.display import display, Markdown
from scipy import integrate
from scipy.integrate import solve_bvp, solve_ivp
from scipy.special import jv
from module_initialize import *

In [2]:
def create_experiment(main_wave: bool, oblique_wave: bool, shear_flow: bool):
    POF = module_POF(main_wave = main_wave, oblique_wave = oblique_wave, shear_flow = shear_flow)
    members = [attr for attr in dir(POF) if not callable(getattr(POF, attr)) and not (attr.startswith("__") or attr.startswith("_"))]
    methods = [attr for attr in dir(POF) if callable(getattr(POF, attr)) and not (attr.startswith("__") or attr.startswith("_"))]

    for member in members if members else []:
        globals()[member] = getattr(POF, member)

    for method in methods if methods else []:
        globals()[method] = getattr(POF, method)
    
    del POF

    return bool(members) or bool(methods)

In [3]:
if_main_wave = True
if_oblique_wave = True
if_shear_flow = True
create_experiment(main_wave = if_main_wave, oblique_wave = if_oblique_wave, shear_flow = if_shear_flow)

True

In [4]:
def get_stokes_drift(verbose = False):

    verbose_print(verbose, 'Initial wave: ')
    verbose_display(verbose, sym.Eq(sym_phi0_symbol, sym_phi0))
    sym_u0x, sym_u0y, sym_u0z = (sym_phi0.diff(var) for var in (sym_x, sym_y, sym_z))

    sym_Us0x = (sym.I * sym_u0y / sym_omega * (sym_u0x).conjugate()).diff(sym_y) + \
                (sym.I * sym_u0z / sym_omega * (sym_u0x).conjugate()).diff(sym_z)
    sym_Us0x = sym_Us0x / 2
    sym_Us0y = 0
    sym_Us0x = sym_Us0x.subs(sym_psi0, - sym.I * sym_omega * sym_h0)
    sym_Us0x = sym_Us0x.simplify()
    
    verbose_display(verbose, sym.Eq(sym.Symbol(r'U^{s0}_x'), sym_Us0x))

    verbose_print(verbose, 'Oblique wave: ')
    verbose_display(verbose, sym.Eq(sym_dphi_symbol, sym_dphi))
    sym_dux, sym_duy, sym_duz = (sym_dphi.diff(var) for var in (sym_x, sym_y, sym_z))

    sym_dUsx = (sym.I * sym_duy / sym_omega * (sym_u0x).conjugate()).diff(sym_y) + \
                (sym.I * sym_duz / sym_omega * (sym_u0x).conjugate()).diff(sym_z) + \
                ((sym.I * sym_u0y / sym_omega).conjugate() * sym_dux).diff(sym_y) + \
                ((sym.I * sym_u0z / sym_omega).conjugate() * sym_dux).diff(sym_z)
    sym_dUsx = sym_dUsx / 2
    sym_dUsy = (sym.I * sym_dux / sym_omega * (sym_u0y).conjugate()).diff(sym_x) + \
                (sym.I * sym_duz / sym_omega * (sym_u0y).conjugate()).diff(sym_z) + \
                ((sym.I * sym_u0x / sym_omega).conjugate() * sym_duy).diff(sym_x) + \
                ((sym.I * sym_u0z / sym_omega).conjugate() * sym_duy).diff(sym_z)
    sym_dUsy = sym_dUsy / 2
    
    sym_dUsx = sym_dUsx.subs(sym_psi0, - sym.I * sym_omega * sym_h0)
    sym_dUsy = sym_dUsy.subs(sym_psi0, - sym.I * sym_omega * sym_h0)

    sym_dUsx = sym_dUsx.simplify()
    sym_dUsy = sym_dUsy.simplify()

    verbose_display(verbose, sym.Eq(sym.Symbol(r'\delta U^s_x'), sym_dUsx))
    verbose_display(verbose, sym.Eq(sym.Symbol(r'\delta U^s_y'), sym_dUsy))

    return (sym_Us0x, sym_Us0y, 0), (sym_dUsx, sym_dUsy, 0)

def get_vortex_force(verbose = False):

    sym_Us0_vec, sym_dUs_vec = get_stokes_drift(verbose = verbose)
     
    sym_vec_O0 = (0, sym_O0, 0)

    sym_dfV_1 = vector_mult(sym_Us0_vec, sym_vec_dO) 
    sym_dfV_2 = vector_mult(sym_dUs_vec, sym_vec_O0)
    sym_dfV = tuple(map(lambda x, y: x + y, sym_dfV_1, sym_dfV_2))
    sym_dfV = simplify_vector(sym_dfV)
    sym_curl_dfV = curl(sym_dfV)
    sym_curl_dfV = simplify_vector(sym_curl_dfV)

    verbose_display(verbose, sym.Eq(sym.Symbol(r'\delta{f^V}_x'), sym_dfV[0]))
    verbose_display(verbose, sym.Eq(sym.Symbol(r'\delta{f^V}_y'), sym_dfV[1]))
    verbose_display(verbose, sym.Eq(sym.Symbol(r'\delta{f^V}_z'), sym_dfV[2]))
    verbose_display(verbose, sym.Eq(sym.Symbol(r'\big(\operatorname{curl}\delta{f^V}\big)_x'), sym_curl_dfV[0]))
    verbose_display(verbose, sym.Eq(sym.Symbol(r'\big(\operatorname{curl}\delta{f^V}\big)_y'), sym_curl_dfV[1]))
    verbose_display(verbose, sym.Eq(sym.Symbol(r'\big(\operatorname{curl}\delta{f^V}\big)_z'), sym_curl_dfV[2]))

    return sym_dfV, sym_curl_dfV

def get_equation_dPsi(verbose = False):

    sym_dfV, sym_curl_dfV = get_vortex_force(verbose = verbose)
    sym_dfVx = sym_dfV[0]
    sym_curl_dfVx = sym_curl_dfV[0]

    laplas = lambda expr: expr.diff(sym_x, 2) + expr.diff(sym_y, 2) + expr.diff(sym_z, 2)

    sym_nu = 0

    # equation (38)
    sym_equation_dOx = (sym_lambda * sym_dOx - sym_nu * laplas(sym_dOx) + sym_V0_symbol * sym_dOx.diff(sym_x)) - \
                        sym_O0 * (sym_dOz + sym_dVx.diff(sym_y)) - sym_curl_dfVx
    sym_equation_dOx = sym_equation_dOx.simplify()

    # equation (39)
    sym_equation_dVx = (sym_lambda * sym_dVx - sym_nu * laplas(sym_dVx) + sym_V0_symbol * sym_dVx.diff(sym_x)) + \
                        sym_dVz * sym_O0 - sym_dfVx
    sym_equation_dVx = sym_equation_dVx.simplify()

    verbose_print(verbose, "Initial equations: ")
    verbose_display(verbose, sym.Eq(sym_equation_dVx, 0))
    verbose_display(verbose, sym.Eq(sym_equation_dOx, 0))

    sym_dVx_to_dPsi = sym.solve(sym_equation_dVx, sym_dVx_symbol)[0]
    sym_dVy_to_dPsi = sym_dPsi.diff(sym_z)
    sym_dVz_to_dPsi = -sym.I * sym_theta * sym_dPsi
    sym_dVx_to_dPsi = sym_dVx_to_dPsi.subs(sym_dVz_symbol, sym_dVz_to_dPsi)

    verbose_display(verbose, sym.Eq(sym_dVx_symbol, sym_dVx_to_dPsi))
    verbose_display(verbose, sym.Eq(sym_dVy_symbol, sym_dVy_to_dPsi))
    verbose_display(verbose, sym.Eq(sym_dVz_symbol, sym_dVz_to_dPsi))

    substitutions_dV_to_dPsi = ((sym_dVx_symbol, sym_dVx_to_dPsi), (sym_dVy_symbol, sym_dVy_to_dPsi), (sym_dVz_symbol, sym_dVz_to_dPsi))
    sym_equation_dPsi = substitute(sym_equation_dOx, substitutions_dV_to_dPsi)
    sym_equation_dPsi = sym_equation_dPsi.simplify()

    over_prefactor = sym.exp(-sym.I * sym_theta * sym_y - sym_lambda * sym_t)
    sym_equation_dPsi = sym_equation_dPsi * over_prefactor
    sym_equation_dPsi = sym_equation_dPsi.simplify()

    sym_equation_dPsi = sym_equation_dPsi.simplify().simplify()
    sym_equation_dPsi = simplify_eq_with_assumptions(sym.Eq(sym_equation_dPsi, 0)).lhs
    sym_equation_dPsi = sym_equation_dPsi.simplify()

    verbose_print(verbose, "Final equation: ")
    verbose_display(verbose, sym.Eq(sym_equation_dPsi, 0))

    return sym_equation_dPsi, substitutions_dV_to_dPsi

def get_main_equations(small_theta_factor = None, verbose = False, super_verbose = False):

    # Printing global dimensionless substitutions
    if super_verbose: print_substitutions(substitutions_dimensionless)

    # Getting dPsi equation
    sym_equation_dPsi, substitutions_dV_to_dPsi = get_equation_dPsi(verbose = super_verbose)
    sym_d2z_dPsi_expr = sym.solve(sym_equation_dPsi, sym_d2z_dPsi)[0]
    verbose_display(verbose, sym.Eq(sym_d2z_dPsi, sym_d2z_dPsi_expr))
    if verbose: print_substitutions(substitutions_dV_to_dPsi)


    # First boundary condition
    verbose_print(verbose, 'First bc: ')
    sym_bc_1 = sym_bc_1_initial
    verbose_display(verbose, sym_bc_1)
    sym_bc_1 = substitute(sym_bc_1, substitutions_dV_to_dPsi).simplify()
    verbose_display(super_verbose, sym_bc_1)
    # Dimensionless substitution
    sym_bc_1 = substitute(sym_bc_1, substitutions_dimensionless)
    sym_bc_1 = sym_bc_1.simplify()
    sym_bc_1 = simplify_eq_with_assumptions(sym.Eq(sym_bc_1, 0)).lhs
    verbose_display(verbose, sym_bc_1)
    verbose_print(verbose)
 
    # Second boundary condition
    verbose_print(verbose, 'Second bc: ')
    sym_bc_2 = sym_bc_2_initial
    verbose_display(verbose, sym_bc_2)
    substitutions_dp = ((sym_dp_symbol, sym_dp), (sym_dp_int_symbol, sym_dp_int))
    sym_bc_2 = substitute(sym_bc_2, substitutions_dp).simplify()
    verbose_display(super_verbose, sym_bc_2)
    sym_bc_2 = substitute(sym_bc_2, substitutions_dV_to_dPsi).simplify()
    verbose_display(super_verbose, sym_bc_2)
    sym_bc_2 = sym_bc_2.replace(sym_d2z_dPsi, sym_d2z_dPsi_expr).simplify()
    sym_bc_2 = simplify_eq_with_assumptions(sym.Eq(sym_bc_2, 0)).lhs
    verbose_display(super_verbose, sym_bc_2)
    # Dimensionless substitution
    sym_bc_2 = substitute(sym_bc_2, substitutions_dimensionless)
    sym_bc_2 = sym_bc_2.simplify()
    verbose_display(super_verbose, sym_bc_2)
    # Getting pressure expression
    sym_dp_int_dim = sym_bc_2.find(sym.Integral)
    sym_dp_int_dim = sym_dp_int_dim.pop() if sym_dp_int_dim else sym.S.Zero
    sym_bc_2 = sym_bc_2.subs(sym_dp_int_dim, sym_dp_int_dim_symbol).simplify()
    sym_bc_2 = simplify_eq_with_assumptions(sym.Eq(sym_bc_2, 0)).lhs
    verbose_display(verbose, sym_bc_2)
    verbose_display(verbose, sym.Eq(sym_dp_int_dim_symbol, sym_dp_int_dim, evaluate = False))
    verbose_print(verbose)

    # dPsi equation dimensionless substitution
    sym_equation_dPsi = substitute(sym_equation_dPsi, substitutions_dimensionless).simplify()
    sym_d2z_dPsi_dim_expr = sym.solve(sym_equation_dPsi, sym_dPsi_dim.diff(sym_z, 2))[0]
    sym_d2z_dPsi_dim_expr = sym_d2z_dPsi_dim_expr.simplify()
    verbose_display(verbose, sym.Eq(sym__d2z_dPsi_dim_symbol, sym_d2z_dPsi_dim_expr))
    

    return sym_bc_1, sym_bc_2, sym_d2z_dPsi_dim_expr

def get_lambda_equation(sym_bc_1, sym_bc_2, verbose = False):
    if not if_oblique_wave: return None

    sym_bc_matrix = sym.linear_eq_to_matrix([sym_bc_1, sym_bc_2], [sym_dh_dim, sym_dpsi_dim])[0]
    sym_eq_lambda = sym_bc_matrix.det()
    sym_eq_lambda = sym_eq_lambda.simplify()
    sym_eq_lambda = simplify_eq_with_assumptions(sym.Eq(sym_eq_lambda, 0)).lhs
    verbose_display(verbose, sym_eq_lambda)

    return sym_eq_lambda



    # # Equation for lambda
        
        # verbose_display(verbose, sym_bc_1)
        

    #     sym_bc_matrix = sym.linear_eq_to_matrix([sym_bc_1, sym_bc_2], [sym_h_dim, sym_psi_dim])[0]
    #     sym_eq_mu = sym_bc_matrix.det()
    #     sym_eq_mu = sym_eq_mu.simplify()
    #     sym_eq_mu = simplify_eq_with_assumptions(sym.Eq(sym_eq_mu, 0))
    #     sym_eq_mu = sym_eq_mu.lhs
    #     sym_eq_mu = sym_eq_mu.simplify()
    #     verbose_display(verbose, sym_eq_mu)
    
    
    # else:
    #     substitutions_dimensionless = None



        

    # return sym_d2z_dPsi_expr, sym_eq_mu, sym_dp_int_dim


In [5]:
sym_bc_1, sym_bc_2, sym_d2z_dPsi_dim_expr = get_main_equations(small_theta_factor = None, verbose = True)

Eq(Derivative(\delta\Psi(z), (z, 2)), \theta*(-\Omega^{\scriptsize{(0)}}*\delta\psi*\lambda*h^{\scriptsize{(0)}}*(sqrt(\theta**2 + 1) + 1)*exp(z*(sqrt(\theta**2 + 1) + 1)) + 2*\Omega^{\scriptsize{(0)}}*\omega*\theta*h^{\scriptsize{(0)}}**2*\delta\Psi(z)*exp(2*z) + \lambda**2*\theta*\delta\Psi(z))/\lambda**2)

Substitutions: 


Eq(\delta{V}_x(z), I*\Omega^{\scriptsize{(0)}}*\theta*\delta\Psi(z)/\lambda)

Eq(\delta{V}_y(z), Derivative(\delta\Psi(z), z))

Eq(\delta{V}_z(z), -I*\theta*\delta\Psi(z))

...


Second bc: 


\delta\psi*(\lambda - I*\omega) + \delta\xi*g - \delta{p}^{\varpi}

2*I*\alpha*\epsilon*\overline{\delta{p}}^{\varpi}_{int}*\theta**2*\tilde{\delta\psi}*h^{\scriptsize{(0)}}**2*(\epsilon + 2*I*\tilde{\lambda}) + 8*I*\alpha*\epsilon*\theta*\tilde{\delta\psi}*\tilde{\lambda}**2*Derivative(\tilde{\delta\Psi}(z), z) + \tilde{\lambda}**3*(\epsilon + 4)*(2*I*\epsilon*\tilde{\delta\psi} - sqrt(\theta**2 + 1)*(4*\tilde{\delta h} + \tilde{\delta\psi}*(4*\tilde{\lambda} + I*(\epsilon - 4))))

Eq(\overline{\delta{p}}^{\varpi}_{int}, Integral((\epsilon*\theta*\tilde{\delta\Psi}(z)*exp(z) - 4*\theta*\tilde{\delta\Psi}(z)*exp(z) + 2*\tilde{\lambda}*sqrt(\theta**2 + 1)*exp(z*sqrt(\theta**2 + 1)) + 2*\tilde{\lambda}*exp(z*sqrt(\theta**2 + 1)))*exp(2*z)*exp(z*sqrt(\theta**2 + 1)), (z, -oo, 0)))

Eq(Derivative(\tilde{\delta\Psi}(z), (z, 2)), -\theta*(\epsilon*\theta*h^{\scriptsize{(0)}}**2*(\epsilon - 4)*\tilde{\delta\Psi}(z)*exp(2*z) + 2*\epsilon*\tilde{\lambda}*h^{\scriptsize{(0)}}**2*(sqrt(\theta**2 + 1) + 1)*exp(z*(sqrt(\theta**2 + 1) + 1)) - 4*\theta*\tilde{\lambda}**2*\tilde{\delta\Psi}(z))/(4*\tilde{\lambda}**2))

In [6]:
sym_eq_lambda = get_lambda_equation(sym_bc_1, sym_bc_2, verbose = True)

-2*\alpha*\epsilon**3*\overline{\delta{p}}^{\varpi}_{int}*\theta**2*h^{\scriptsize{(0)}}**2 + 4*I*\alpha*\epsilon**2*\overline{\delta{p}}^{\varpi}_{int}*\theta**2*\tilde{\lambda}*h^{\scriptsize{(0)}}**2 + 8*\alpha*\epsilon**2*\overline{\delta{p}}^{\varpi}_{int}*\theta**2*h^{\scriptsize{(0)}}**2 - 8*\alpha*\epsilon**2*\theta*\tilde{\lambda}**2*Derivative(\tilde{\delta\Psi}(z), z) - 16*\alpha*\epsilon*\overline{\delta{p}}^{\varpi}_{int}*\theta**2*\tilde{\lambda}**2*h^{\scriptsize{(0)}}**2 + 16*I*\alpha*\epsilon*\overline{\delta{p}}^{\varpi}_{int}*\theta**2*\tilde{\lambda}*h^{\scriptsize{(0)}}**2 + 16*\alpha*\epsilon*\theta*\tilde{\lambda}**3*sqrt(-\theta**2 - 1)*Derivative(\tilde{\delta\Psi}(z), z) + 32*I*\alpha*\epsilon*\theta*\tilde{\lambda}**3*Derivative(\tilde{\delta\Psi}(z), z) + 32*\alpha*\epsilon*\theta*\tilde{\lambda}**2*Derivative(\tilde{\delta\Psi}(z), z) + 64*\alpha*\theta*\tilde{\lambda}**3*sqrt(-\theta**2 - 1)*Derivative(\tilde{\delta\Psi}(z), z) + \epsilon**3*\tilde{\lambda

In [8]:
sym_eq_lambda_CL = sym_eq_lambda.replace(sym_alpha, 0).simplify()

In [9]:
sym_eq_lambda_CL

\tilde{\lambda}**3*(\epsilon**3*sqrt(\theta**2 + 1) - 2*\epsilon**3 - 8*\epsilon**2*\tilde{\lambda}*sqrt(-\theta**2 - 1) + 8*I*\epsilon**2*\tilde{\lambda} - 4*\epsilon**2*sqrt(\theta**2 + 1) - 16*\epsilon*\theta**2 - 16*\epsilon*\tilde{\lambda}**2*sqrt(\theta**2 + 1) + 32*I*\epsilon*\tilde{\lambda} - 16*\epsilon*sqrt(\theta**2 + 1) + 16*\epsilon - 64*\theta**2 - 64*\tilde{\lambda}**2*sqrt(\theta**2 + 1) + 128*\tilde{\lambda}*sqrt(-\theta**2 - 1) + 64*sqrt(\theta**2 + 1) - 64)

In [11]:
def check_alpha_zero_epsilon_zero():
    sym_bc_1_alpha_zero = sym_bc_1.replace(sym_alpha, 0)
    sym_bc_2_alpha_zero = sym_bc_2.replace(sym_alpha, 0)
    display(sym_bc_1_alpha_zero)
    display(sym_bc_2_alpha_zero)
    print()
    sym_bc_2_alpha_zero = sym_bc_2_alpha_zero.args[-1]
    sym_bc_1_alpha_zero = sym_bc_1_alpha_zero.args[-1]
    display(sym_bc_1_alpha_zero)
    display(sym_bc_2_alpha_zero)
    print()
    sym_bc_1_alpha_zero_eps_zero = sym_bc_1_alpha_zero.replace(sym_epsilon, 0)
    sym_bc_2_alpha_zero_eps_zero = sym_bc_2_alpha_zero.replace(sym_epsilon, 0)
    display(sym_bc_1_alpha_zero_eps_zero)
    display(sym_bc_2_alpha_zero_eps_zero)